In [85]:
import pandas as pd

In [86]:
clients = pd.read_excel('data/top_clients.xlsx', sheet_name=1)
base = pd.read_csv('Duracion Envio Solicitudes - clientes.csv')

In [87]:
clients.head()

,CLIENTE,COMERCIAL
0,FCM SA,TEFY
1,C.I. GLOBALCARIBE INTERNACIONAL SAS,SHARON
2,GOZ IMPORT AND EXPORT SAS,SHARON
3,CUATRO VIENTOS CAFÉ LLC,GABO
4,KOMERCADO SAS,ANDRES


In [88]:
# import pandas as pd
# import re
# from unidecode import unidecode
# from rapidfuzz import process, fuzz

# def clean_name(s: str) -> str:
#     s = unidecode(str(s))
#     s = s.lower().strip()
#     s = re.sub(r'[^a-z0-9\s]', '', s)
#     return re.sub(r'\s+', ' ', s)

# def enforce_top_clients_match(
#     df_top: pd.DataFrame,
#     df_db: pd.DataFrame,
#     top_key: str = 'cliente',
#     db_key: str = 'cliente',
#     threshold: int = 90
# ) -> pd.DataFrame:
#     """
#     1. Limpia nombres en ambas tablas.
#     2. Merge exacto → marca los que empatan.
#     3. Para los no-empates, busca el mejor fuzzy match.
#     4. Si alguno queda con score < threshold, levanta ValueError.
#     5. Devuelve df_top con una columna 'official_name' mapeada de la DB.
#     """
#     # 1) Claves limpias
#     df_top = df_top.copy()
#     df_db  = df_db.copy()
#     df_top['_key'] = df_top[top_key].map(clean_name)
#     df_db['_key']  = df_db[db_key].map(clean_name)

#     # 2) Exact match
#     merged = df_top.merge(
#         df_db[['_key', db_key]],
#         on='_key', how='left', suffixes=('', '_db'),
#         indicator=True
#     )
#     # Aquellos que empatan exacto usan el nombre de DB
#     merged['official_name'] = merged.loc[merged['_merge']=='both', db_key]

#     # 3) Fuzzy sólo para los left_only
#     no_match = merged[merged['_merge']=='left_only']
#     if not no_match.empty:
#         choices = df_db['_key'].tolist()
#         # obtiene mejor match y score
#         def find_best(key):
#             m, score, _ = process.extractOne(key, choices, scorer=fuzz.WRatio)
#             # return m, score

#         suggestions = no_match['_key'].map(find_best)
#         no_match[['suggested_key','score']] = pd.DataFrame(suggestions.tolist(), index=no_match.index)
#         no_match = no_match.merge(
#             df_db[['_key', db_key]],
#             left_on='suggested_key', right_on='_key',
#             how='left',
#             suffixes=('','_db2')
#         )
#         # 4) falla si algún score < threshold
#         low_conf = no_match[no_match['score'] < threshold]
#         if not low_conf.empty:
#             bad = low_conf[top_key].tolist()
#             raise ValueError(
#                 f"Los siguientes clientes no alcanzan un match confiable (score < {threshold}):\n"
#                 + "\n".join(f"- {c}" for c in bad)
#             )
#         # Asigna la sugerencia para los demás
#         merged.loc[no_match.index, 'official_name'] = no_match[f'{db_key}_db2']

#     # 5) Verificación final
#     if merged['official_name'].isna().any():
#         faltan = merged[merged['official_name'].isna()][top_key].tolist()
#         raise RuntimeError(f"Quedaron sin match estas filas: {faltan}")

#     return merged.drop(columns=['_key','_merge','suggested_key','score'])

# # — Ejemplo de uso —
# clients = pd.read_excel('data/top_clients.xlsx')
# base = pd.read_csv('Duracion Envio Solicitudes - clientes.csv')
# clients.columns = clients.columns.str.strip().str.lower()
# base   .columns = base  .columns.str.strip().str.lower()
# df_validado = enforce_top_clients_match(clients, base, threshold=90)
# # Ahora df_validado['official_name'] alinea 1:1 con tu base de datos.


In [89]:
clients.columns = clients.columns.str.strip().str.lower()
clients['comercial'].unique()

array(['TEFY', 'SHARON', 'GABO', 'ANDRES', 'CALVIN', 'PEDRO', 'IRINA',
       'FELIPE', 'NEGRO', 'FARAH', 'ELIA', 'RENE', 'JAIME', 'TILCIA'],
      dtype=object)

In [90]:
comercial_map = {
    'TEFY': 'Stephanie Bruges',
    'PEDRO': 'Pedro Luis Bruges',
    'SHARON': 'Sharon Zuñiga',
    'FARAH': 'Johnny Farah',
    'ANDRES': 'Andrés Consuegra',
    'GABO': 'Comercial Generico',
    'CALVIN': 'Ivan Zuluaga',
    'NEGRO': 'Jorge Sánchez',
    'IRINA': 'Irina Paternina',
    'ELIA': 'Comercial Generico',
    'FELIPE': 'Felipe Hoyos',
    'TILCIA': 'Comercial Generico',
    'RENE': 'Comercial Generico',
    'JAIME': 'Jaime Ahumada',
}

# 3) Aplica el mapeo; los que no estén en el dict quedan igual
clients['comercial'] = clients['comercial'].map(comercial_map) \
                             .fillna(clients['comercial'])

print(clients['comercial'].unique())

['Stephanie Bruges' 'Sharon Zuñiga' 'Comercial Generico'
 'Andrés Consuegra' 'Ivan Zuluaga' 'Pedro Luis Bruges' 'Irina Paternina'
 'Felipe Hoyos' 'Jorge Sánchez' 'Johnny Farah' 'Jaime Ahumada']


In [91]:
clients.head()

,cliente,comercial
0,FCM SA,Stephanie Bruges
1,C.I. GLOBALCARIBE INTERNACIONAL SAS,Sharon Zuñiga
2,GOZ IMPORT AND EXPORT SAS,Sharon Zuñiga
3,CUATRO VIENTOS CAFÉ LLC,Comercial Generico
4,KOMERCADO SAS,Andrés Consuegra


In [92]:
clients = clients.reset_index(drop=True) 
clients.insert(
    1,                                           
    'ranking',                                       
    range(1, len(clients) + 1)                 
)

In [93]:
nuevo_orden = ['ranking', 'cliente', 'comercial']

clients = clients[nuevo_orden]


In [94]:
clients.head()

,ranking,cliente,comercial
0,1,FCM SA,Stephanie Bruges
1,2,C.I. GLOBALCARIBE INTERNACIONAL SAS,Sharon Zuñiga
2,3,GOZ IMPORT AND EXPORT SAS,Sharon Zuñiga
3,4,CUATRO VIENTOS CAFÉ LLC,Comercial Generico
4,5,KOMERCADO SAS,Andrés Consuegra


In [97]:
clients.to_excel('data/top_clientes.xlsx', index=False)